## Import 1: Classes & Path

In [ ]:
# Import 1. 
# Import functions.py into notebook from src folder. 
import sys

# Path relative to your notebook.
sys.path.append("/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/src")

# Import my .py classes.
import data_clean
import plots

# Auto reload.
%load_ext autoreload
%autoreload 2

## Import 2: All Libs

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

## Read File & Inital Drop of Columns

In [ ]:
# Read file in.
# Dropping columns to trim data to 1996-2022.
# Use clean class and init a cleaner object.
cleaner = data_clean.Clean()

niger = cleaner.read_file('/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/data/Niger.csv')
niger = niger.drop(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995'], axis=1)

# Make a metadata df for indicator name, source note, and source_org just in case.
niger_meta = cleaner.read_file('/Users/joel/Desktop/daimil10/CAPSTONE_2/Niger-Political-Shift-Examination/data/Metadata_Indicator_Descriptors.csv')

niger.head()

## Describing the Dataset

In [ ]:
# Describing the Dataset 1.
niger.describe()

In [ ]:
# Describing the Dataset 2.
niger.info()

In [ ]:
# Describing the Dataset 3.
niger.index

In [ ]:
# Describing the Dataset 4.
niger.columns

In [ ]:
# Describing the Dataset 5.
missing_v = niger.isnull().sum()
missing_v

In [ ]:
# Describing the Dataset 6.
# Using the function that spits outdict of col names and dtypes. 
cleaner.get_column_features(niger)

## Project Goal #1:
Examine overall World Governance Index from 1996 to 2022 (26 year span).
- Political Stability and Absence of Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence, including terrorism. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

- Control of Corruption captures perceptions of the extent to which public power is exercised for private gain, including both petty and grand forms of corruption, as well as "capture" of the state by elites and private interests. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

- Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy formulation and implementation, and the credibility of the government's commitment to such policies. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

- Regulatory Quality captures perceptions of the ability of the government to formulate and implement sound policies and regulations that permit and promote private sector development. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

- Voice and Accountability captures perceptions of the extent to which a country's citizens are able to participate in selecting their government, as well as freedom of expression, freedom of association, and a free media. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

- Rule of Law captures perceptions of the extent to which agents have confidence in and abide by the rules of society, and in particular the quality of contract enforcement, property rights, the police, and the courts, as well as the likelihood of crime and violence. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

In [ ]:
# Slice DF by PV.EST, CC.EST, GE.EST, RQ.EST, VA.EST, RL.EST indicator code which all make up the world governance index.
indicator_codes = ['PV.EST', 'CC.EST', 'GE.EST']
PG1 = niger[niger['Indicator Code'].isin(indicator_codes)]

# Using replace for the labels since they are kinda long
PG1['Indicator Name'] = PG1['Indicator Name'].replace('Political Stability and Absence of Violence/Terrorism: Estimate','Political Stability')
PG1['Indicator Name'] = PG1['Indicator Name'].replace('Control of Corruption: Estimate','Control of Corruption')
PG1['Indicator Name'] = PG1['Indicator Name'].replace('Government Effectiveness: Estimate','Government Effectiveness')

# No Data for 1997, 1999, 2001, 2022
PG1 = PG1.drop(['1997', '1999', '2001', '2022'], axis=1)

# Use the style that i like.
style.use('fivethirtyeight')

# Figsize.
plt.figure(figsize=(16, 9))

# Plot each indicator code separately.
for indicator in indicator_codes:
    # Not to repeat.
    if indicator in PG1['Indicator Code'].unique():
        # Using indicator code to cycle through
        indicator_data = PG1[PG1['Indicator Code'] == indicator]
        # X = Year, Y= Aggregated Indicator Scale.
        # values.flatten for the list of values.
        # F string for label, used in last project.
        plt.plot(indicator_data.columns[4:], indicator_data.iloc[:, 4:].values.flatten(), marker='o', label=f'{indicator_data["Indicator Name"].iloc[0]}')

# Label and title.
plt.xlabel('Year', color='green')
plt.ylabel('Aggregate Indicator', color='green')
plt.title('World Governance Index For Niger', color='green')

#Label for Coup: Change of Power.
plt.plot([], [], ' ', label="Coup: Change of Power")

#Label for Attempted Coup.
plt.plot([], [], ' ', label="Attempted Coup")

# Set the background color to white.
plt.gca().set_facecolor('grey')

# Using all the years present for the xticks.
# Set the tick color to black.
plt.xticks(rotation=45)
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')

# Show the legend.
plt.legend()

# plt.show.
plt.show()

## Project Goal #2:
Explore economic down turn over specified time frame.

- GDP per capita: is gross domestic product divided by midyear population. GDP is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources. Data are in current U.S. dollars.

- General government expenditure on education: (current, capital, and transfers) is expressed as a percentage of GDP. It includes expenditure funded by transfers from international sources to government. General government usually refers to local, regional and central governments.

- Labor force participation rate (Male/Female): is the proportion of the population ages 15 and older that is economically active: all people who supply labor for the production of goods and services during a specified period.

- Public and publicly guaranteed debt comprises long-term external obligations of public debtors, including the national government,  Public Corporations, State Owned Enterprises, Development Banks and Other Mixed Enterprises, political subdivisions (or an agency of either), autonomous public bodies, and external obligations of private debtors that are guaranteed for repayment by a public entity. Data are in current U.S. dollars.

In [ ]:
# For some reason some of the indicator codes were not populating.
# Made me do the long way istead. 
a = niger[niger['Indicator Code'] == 'SL.TLF.CACT.FE.ZS']
b = niger[niger['Indicator Code'] == 'NY.GDP.PCAP.CD']
c = niger[niger['Indicator Code'] == 'SL.TLF.CACT.MA.ZS']
d = niger[niger['Indicator Code'] == 'SE.XPD.TOTL.GD.ZS']
e = niger[niger['Indicator Code'] == 'DT.DOD.DPPG.CD']

# Combine them vertically (stacked like).
PG2 = pd.concat([a, b, c, d, e], axis=0)

# Reset the index of the combined data frame (cause it was funky).
PG2 = PG2.reset_index(drop=True)


# Figsize.
# Using series.plot show in Week 4 of learn material.  
plt.figure(figsize=(16, 12))
series.plot(kind='bar', stacked=True)

# Add lables and title.
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(title)

# Roatate xticks.
plt.xticks(rotation= 45)

# Set the background color to black.
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.gca.html helped a ton.. 
plt.gca().set_facecolor('black')

# Wanted to get the legend in the bottom left corner.. 
# https://stackoverflow.com/questions/25068384/bbox-to-anchor-and-loc-in-matplotlib
plt.legend(title='Method', bbox_to_anchor=(1.05, 0), loc='lower left')

# plt.show.
plt.show()


## Project Goal #3:
Highlight Political Regime Metrics (multiple coups and multiple leaders). 

## Project Goal #4:
Uncover data points related to corruption perception index & economic indicators. 


## Project Goal #5:
Hypo test (1, 2, 3, 4, or 5)

## Project Goal #6:
Linear Model (1, 2, or 3)